In [1]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import dotenv_values
import random

In [2]:
random.seed(42) 

In [3]:
config = dotenv_values('.env')

# Fn

In [4]:
def fix_tracks(tracks: list) -> list:
    "Sacamos cosas posteriores al guión y capitalizamos"
    # sacamos cosas como " - Remix" o " - Edit"
    tracks = [item["track"]["name"].split(" -")[0] for item in tracks if item["track"]]
    tracks = [t.capitalize() for t in tracks]
    return tracks


In [5]:
def long_weird_track_names(tracks: list, max_length: int=30) -> None:
    """
    Devuelve tracks si superan longitud o tienen 'edit' o '(' en el nombre.
    """
    long_names = []
    for item in tracks:
        if len(item) > max_length or 'edit' in item or '(' in item:
            long_names.append(item)
    print('Estos tracks quizá son largos'.center(80, '-'))
    [print(t) for t in long_names]

In [6]:
def replace_names(tracks: list, replacements: dict) -> list:
    """
    Reemplaza nombres de canciones y . Si no está en el diccionario, se mantiene tal cual.
    """
    tracks = [replacements.get(t, t).capitalize() for t in tracks]
    return tracks


In [7]:
def get_tracks(url: str) -> list[str]:
    """
    Obtiene los tracks de una playlist de Spotify.
    """
    try:
        scope = "playlist-read-private"
        auth_manager = SpotifyOAuth(
            client_id=config["SPOTIFY_CLIENT_ID"],
            client_secret=config["SPOTIFY_CLIENT_SECRET"],
            redirect_uri=config["SPOTIPY_REDIRECT_URI"],
            scope=scope,
            cache_path=".spotifycache",  # This will store the token for future use
        )
        sp = spotipy.Spotify(auth_manager=auth_manager)

        results = sp.playlist_items(url)
        tracks = results["items"]
        # Spotify devuelve resultados paginados (de a 100), nos aseguramos de traerlos todos
        while results["next"]:
            results = sp.next(results)
            tracks.extend(results["items"])

        tracks = fix_tracks(tracks)
        print(f"Se encontraron {len(tracks)} tracks")
        return tracks
    except Exception as e:
        print(f"Error al conectar con Spotify: {e}")
        return []


# Tracks

In [8]:
tracks = get_tracks(config["SPOTIFY_PLAYLIST_URL"])
with open("tracks.txt", "w") as f:
    for track in tracks:
        f.write(f"{track}\n")

Se encontraron 75 tracks


In [9]:
a = "Rocket man (i think it's going to be a long, long time)"
'(' in a 

True

In [10]:
long_weird_track_names(tracks)

-------------------------Estos tracks quizá son largos--------------------------
Igual que un ángel (with peso pluma)
Loco (tu forma de ser)
Un'estate italiana (notti magiche)
Rocket man (i think it's going to be a long, long time)
Slam dunk (da funk)
Praise you (radio edit)
All night long (all night)


In [11]:
replacements = {
    "Igual que un ángel (with peso pluma)": "Igual Que Un Ángel",
    "Rocket man (i think it's going to be a long, long time)": "Rocket Man",
    "All night long (all night)": "All Night Long",
    "Un'estate italiana (notti magiche)": "Un'Estate Italiana",
    "Praise you (radio edit)": "Praise You",
}

In [12]:
tracks = replace_names(tracks, replacements)

# Cartones

In [13]:
NUM_CARTONES = 24
NUM_TRACKS_POR_CARTON = 15

In [14]:
def sampleTracks(tracks, sampleSize):
    """
    Genera un carton con tracks aleatorios
    """
    return random.sample(tracks, sampleSize)

In [15]:
def generar_cartones(
    tracks: list, 
    numCartones: int = NUM_CARTONES, 
    tracksPorCarton: int = NUM_TRACKS_POR_CARTON
) -> dict:
    """
    Genera M cartones con N tracks aleatorios
    """
    if len(tracks) < tracksPorCarton:
        raise ValueError(
            f"No hay suficientes tracks ni para un cartón (hay {len(tracks)} disponibles, se necesitan {tracksPorCarton})"
        )

    cartones = dict()
    for i in range(1, numCartones + 1):
        cartones[i] = sampleTracks(tracks, sampleSize=tracksPorCarton)
        # Evitamos cartones iguales a otros
        if i > 1:
            for j in range(1, i):
                while len(set(cartones[i]) & set(cartones[j])) == tracksPorCarton:
                    cartones[i] = sampleTracks(tracks, sampleSize=tracksPorCarton)
    return cartones

In [16]:
cartones = generar_cartones(tracks, numCartones=NUM_CARTONES, tracksPorCarton=NUM_TRACKS_POR_CARTON)

# PDF

In [17]:
fontName = "Chalkboard"
pdfmetrics.registerFont(TTFont(fontName, f'{fontName}.ttc'))

In [18]:
def crear_pdf_cartones(
    cartones: dict, 
    filename: str = "bingo_musical.pdf", 
    cartones_por_pagina=2,
    filas=5,
    columnas=3
):
    """
    Crea un PDF con los cartones para imprimir
    Grilla parametrizable
    """
    doc = SimpleDocTemplate(
        filename,
        pagesize=A4,
        topMargin=1.2 * cm,
        bottomMargin=1.2 * cm,
        leftMargin=1 * cm,
        rightMargin=1 * cm,
    )

    elementos = []
    styles = getSampleStyleSheet()

    # Estilo personalizado para el título
    titulo_style = ParagraphStyle(
        "TituloCarton",
        parent=styles["Heading2"],
        alignment=1,  # Centrado
        fontSize=20,
        spaceAfter=0.6 * cm,
        textColor=colors.black,
        fontName="Helvetica-Bold",
        # borderWidth=1,
        # borderColor=colors.black,
        # borderPadding=8,
        # backColor=colors.white,
    )

    # Estilo para las canciones
    cancion_style = ParagraphStyle(
        "Cancion",
        parent=styles["Normal"],
        fontSize=10,
        alignment=1,  # Centrado
        leading=11,
        wordWrap="CJK",  # Permite mejor wrap de texto
        fontName=fontName,
        textColor=colors.black,
    )

    # Procesar cada cartón
    cartones_procesados = 0
    for numero_carton, tracks in cartones.items():
        # Título del cartón
        titulo = Paragraph(f"CARTÓN #{numero_carton}", titulo_style)
        elementos.append(titulo)

        # Organizar tracks en matriz filas x columnas
        tabla_data = []
        for fila in range(filas):
            fila_data = []
            for col in range(columnas):
                idx = fila * columnas + col
                if idx < len(tracks):
                    track = tracks[idx]
                    # No acortamos los nombres, los dejamos completos
                    fila_data.append(Paragraph(track, cancion_style))
                else:
                    fila_data.append("")
            tabla_data.append(fila_data)

        # Calcular ancho de columna para que ocupe todo el ancho disponible
        ancho_disponible = A4[0] - (2 * cm) # Ancho página - márgenes izq/der
        ancho_columna = ancho_disponible / columnas

        # Crear tabla con dimensiones apropiadas
        tabla = Table(
            tabla_data, 
            colWidths=[ancho_columna] * columnas, 
            rowHeights=[2.2 * cm] * filas
        )
        
        tabla.setStyle(
            TableStyle(
                [
                    # Fondo blanco para todas las celdas
                    ("BACKGROUND", (0, 0), (-1, -1), colors.white),
                    ("TEXTCOLOR", (0, 0), (-1, -1), colors.black),
                    ("ALIGN", (0, 0), (-1, -1), "CENTER"),
                    ("VALIGN", (0, 0), (-1, -1), "MIDDLE"),
                    ("FONTNAME", (0, 0), (-1, -1), "Helvetica"),
                    ("FONTSIZE", (0, 0), (-1, -1), 9),
                    # Bordes externos más gruesos para el marco
                    (
                        "LINEBELOW",
                        (0, -1), # Última fila
                        (-1, -1),
                        2,
                        colors.black,
                    ),  # Borde inferior grueso
                    (
                        "LINEABOVE",
                        (0, 0),
                        (-1, 0),
                        2,
                        colors.black,
                    ),  # Borde superior grueso
                    (
                        "LINEBEFORE",
                        (0, 0),
                        (0, -1),
                        2,
                        colors.black,
                    ),  # Borde izquierdo grueso
                    (
                        "LINEAFTER",
                        (-1, 0),
                        (-1, -1),
                        2,
                        colors.black,
                    ),  # Borde derecho grueso
                    # Bordes internos más delgados y elegantes
                    ("INNERGRID", (0, 0), (-1, -1), 0.5, colors.grey),
                    # Padding generoso para que se vea espacioso
                    ("LEFTPADDING", (0, 0), (-1, -1), 8),
                    ("RIGHTPADDING", (0, 0), (-1, -1), 8),
                    ("TOPPADDING", (0, 0), (-1, -1), 6),
                    ("BOTTOMPADDING", (0, 0), (-1, -1), 6),
                    # Sombra sutil en los bordes (simulada con bordes adicionales)
                    ("BOX", (0, 0), (-1, -1), 2, colors.black),
                ]
            )
        )

        elementos.append(tabla)
        cartones_procesados += 1

        # Agregar espacio entre cartones o salto de página
        if cartones_procesados % cartones_por_pagina == 0 and cartones_procesados < len(
            cartones
        ):
            elementos.append(PageBreak())
        else:
            elementos.append(Spacer(1, 0.8 * cm))

    # Generar PDF
    doc.build(elementos)
    print(f"PDF generado: {filename}")

In [19]:
crear_pdf_cartones(cartones, "bingo_cumple.pdf")

PDF generado: bingo_cumple.pdf


In [20]:
tracksUsados = set()
for _, t in cartones.items():
    tracksUsados.update(t)

In [21]:
if set(tracks) == tracksUsados:
    print("Se usaron todos los tracks de la playlist.")
else:
    tracks_no_usados = set(tracks) - set(tracksUsados)
    print("No se usaron todos los tracks de la playlist.")
    print(f"Tracks no usados: {tracks_no_usados}")

Se usaron todos los tracks de la playlist.
